In [10]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [1]:
!pip -q install smplx trimesh pyrender imageio imageio-ffmpeg pyglet
!apt-get -qq update
!apt-get -qq install -y ffmpeg libgl1-mesa-glx libegl1-mesa


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.0/737.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libegl1-mesa:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../libegl1-mesa_23.0.4-0ubuntu1~22.04.1_amd64.deb ...
Unpacking libegl1-mesa:amd64 (23.0.4-0ubuntu1~22.04.1) ...
Setting up libegl1-mesa:amd64 (23.0.4-0ubuntu1~22.04.1) ...


In [11]:
SMPLX_MODEL_DIR = "/content/drive/MyDrive/OSXBackup/OSX/common/utils/human_model_files"


In [14]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

SMPLX_MODEL_DIR = "/content/drive/MyDrive/OSXBackup/human_models_folder"

layer = smplx.create(
    SMPLX_MODEL_DIR,
    model_type="smplx",
    gender="neutral",
    use_pca=False,
    batch_size=1,
    create_global_orient=True,
    create_body_pose=True,
    create_betas=True,
    create_expression=True,
    create_jaw_pose=True,
    create_left_hand_pose=True,
    create_right_hand_pose=True,
).to(device).eval()

faces = layer.faces
print("Loaded SMPL-X. faces:", faces.shape)


device: cpu
Loaded SMPL-X. faces: (20908, 3)


In [19]:
import numpy as np
import torch
import smplx

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# --- load predicted motion ---
MOTION_NPZ = "/content/motion_20s_pred.npz"
z = np.load(MOTION_NPZ, allow_pickle=True)

poses = z["poses"].astype(np.float32)          # (T,55,3)
expr  = z["expressions"].astype(np.float32)    # (T,10)
betas = z["betas"].astype(np.float32)          # (10,)
fps   = int(z["fps"])
transl = z["transl"].astype(np.float32)        # (T,3)

# --- choose how many seconds to render ---
RENDER_SECONDS = 10
T = min(poses.shape[0], int(RENDER_SECONDS * fps))
print("Rendering frames:", T)

# --- recreate SMPL-X with batch_size=T ---
SMPLX_MODEL_DIR = "/content/drive/MyDrive/OSXBackup/OSX/common/utils/human_model_files"
layer = smplx.create(
    model_path=SMPLX_MODEL_DIR,
    model_type="smplx",
    gender="neutral",
    use_pca=False,
    batch_size=T,                     # <-- critical
    create_global_orient=True,
    create_body_pose=True,
    create_betas=True,
    create_expression=True,
    create_jaw_pose=True,
    create_leye_pose=True,
    create_reye_pose=True,
    create_left_hand_pose=True,
    create_right_hand_pose=True,
).to(device).eval()

faces = layer.faces

# --- split packed poses (SMPL-X 55 joints) ---
global_orient = poses[:T, 0, :]                         # (T,3)
body_pose     = poses[:T, 1:22, :].reshape(T, -1)       # (T,63)
jaw_pose      = poses[:T, 22, :]                        # (T,3)
leye_pose     = poses[:T, 23, :]                        # (T,3)
reye_pose     = poses[:T, 24, :]                        # (T,3)

left_hand     = poses[:T, 25:40, :].reshape(T, -1)      # (T,45)
right_hand    = poses[:T, 40:55, :].reshape(T, -1)      # (T,45)

betas_T = np.repeat(betas.reshape(1, 10), T, axis=0)    # (T,10)
transl_T = transl[:T]                                   # (T,3)

# --- to torch ---
global_orient_t = torch.from_numpy(global_orient).float().to(device)
body_pose_t     = torch.from_numpy(body_pose).float().to(device)
jaw_pose_t      = torch.from_numpy(jaw_pose).float().to(device)
leye_pose_t     = torch.from_numpy(leye_pose).float().to(device)
reye_pose_t     = torch.from_numpy(reye_pose).float().to(device)

expr_t          = torch.from_numpy(expr[:T]).float().to(device)
left_hand_t     = torch.from_numpy(left_hand).float().to(device)
right_hand_t    = torch.from_numpy(right_hand).float().to(device)

betas_t         = torch.from_numpy(betas_T).float().to(device)
transl_t        = torch.from_numpy(transl_T).float().to(device)

print("Shapes into SMPL-X:",
      global_orient_t.shape, body_pose_t.shape, jaw_pose_t.shape,
      leye_pose_t.shape, reye_pose_t.shape,
      left_hand_t.shape, right_hand_t.shape,
      expr_t.shape, betas_t.shape, transl_t.shape)

# --- forward ---
with torch.no_grad():
    out = layer(
        betas=betas_t,
        global_orient=global_orient_t,
        body_pose=body_pose_t,
        jaw_pose=jaw_pose_t,
        leye_pose=leye_pose_t,
        reye_pose=reye_pose_t,
        left_hand_pose=left_hand_t,
        right_hand_pose=right_hand_t,
        expression=expr_t,
        transl=transl_t,
        return_verts=True
    )

verts = out.vertices.detach().cpu().numpy()  # (T,V,3)
print("verts:", verts.shape)


device: cpu
Rendering frames: 300
Shapes into SMPL-X: torch.Size([300, 3]) torch.Size([300, 63]) torch.Size([300, 3]) torch.Size([300, 3]) torch.Size([300, 3]) torch.Size([300, 45]) torch.Size([300, 45]) torch.Size([300, 10]) torch.Size([300, 10]) torch.Size([300, 3])
verts: (300, 10475, 3)


In [21]:
!apt-get -qq update
!apt-get -qq install -y libegl1-mesa libgl1-mesa-glx mesa-utils xvfb ffmpeg
!pip -q install pyrender imageio imageio-ffmpeg trimesh pyglet


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package mesa-utils-bin:amd64.
(Reading database ... 117534 files and directories currently installed.)
Preparing to unpack .../mesa-utils-bin_8.4.0-1ubuntu1_amd64.deb ...
Unpacking mesa-utils-bin:amd64 (8.4.0-1ubuntu1) ...
Selecting previously unselected package mesa-utils.
Preparing to unpack .../mesa-utils_8.4.0-1ubuntu1_amd64.deb ...
Unpacking mesa-utils (8.4.0-1ubuntu1) ...
Setting up mesa-utils-bin:amd64 (8.4.0-1ubuntu1) ...
Setting up mesa-utils (8.4.0-1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...


In [22]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl"


In [23]:
import numpy as np
import trimesh
import pyrender
import imageio

def render_video_egl(verts, faces, fps, out_mp4="face_motion.mp4", img_size=(640, 640)):
    scene = pyrender.Scene(bg_color=[255, 255, 255, 255], ambient_light=[0.3, 0.3, 0.3])

    # Create mesh
    mesh_tr = trimesh.Trimesh(vertices=verts[0], faces=faces, process=False)
    mesh = pyrender.Mesh.from_trimesh(mesh_tr, smooth=True)
    mesh_node = scene.add(mesh)

    # Camera: closer + slightly up for more face focus
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    cam_pose = np.eye(4)
    cam_pose[2, 3] = 1.6   # zoom in (smaller = closer)
    cam_pose[1, 3] = 0.15  # slightly up
    scene.add(camera, pose=cam_pose)

    # Light
    light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
    scene.add(light, pose=cam_pose)

    r = pyrender.OffscreenRenderer(viewport_width=img_size[0], viewport_height=img_size[1])

    writer = imageio.get_writer(out_mp4, fps=fps, codec="libx264", quality=8)

    for i in range(verts.shape[0]):
        # replace mesh
        scene.remove_node(mesh_node)
        mesh_tr = trimesh.Trimesh(vertices=verts[i], faces=faces, process=False)
        mesh = pyrender.Mesh.from_trimesh(mesh_tr, smooth=True)
        mesh_node = scene.add(mesh)

        color, _ = r.render(scene)
        writer.append_data(color)

        if (i + 1) % 50 == 0:
            print(f"Rendered {i+1}/{verts.shape[0]} frames")

    writer.close()
    r.delete()
    return out_mp4

out_mp4 = render_video_egl(verts, faces, fps=fps, out_mp4="face_motion.mp4", img_size=(640,640))
out_mp4


Rendered 50/300 frames
Rendered 100/300 frames
Rendered 150/300 frames
Rendered 200/300 frames
Rendered 250/300 frames
Rendered 300/300 frames


'face_motion.mp4'

With Audio

In [24]:
FULL_AUDIO = "/content/drive/MyDrive/Extracted_Parameters/ch08_speakerview_16k.wav"  # CHANGE THIS
START_SEC = 300    # e.g. start at 5 minutes
DURATION = 20      # seconds


In [25]:
AUDIO_SLICE = "audio_20s.wav"

!ffmpeg -y \
  -ss {START_SEC} \
  -t {DURATION} \
  -i "{FULL_AUDIO}" \
  -ac 1 -ar 16000 \
  "{AUDIO_SLICE}"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [26]:
!ffmpeg -y \
  -i face_motion.mp4 \
  -i audio_20s.wav \
  -c:v copy \
  -c:a aac \
  -shortest \
  face_motion_with_audio.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab